In [38]:
import torch
import torchvision
import torch.nn as nn
from torchvision import datasets,transforms,models
from torchvision.models import EfficientNet_V2_S_Weights, SqueezeNet1_1_Weights, ShuffleNet_V2_X0_5_Weights

In [15]:
model = models.efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.DEFAULT)
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  

In [16]:
num_features = model.classifier[1].in_features
print(num_features)
model.classifier[1] = nn.Linear(num_features, 2)

1280


In [17]:
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  

In [18]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 20,180,050 trainable parameters


In [28]:
sq_model = models.squeezenet1_1(weights=SqueezeNet1_1_Weights.DEFAULT)

In [29]:
print(f"The squeezenet model has {count_parameters(sq_model):,} trainable parameters")

The squeezenet model has 1,235,496 trainable parameters


In [30]:
#freezing previous layers
for param in sq_model.features.parameters():
    param.requires_grad = False

In [31]:
# modifying the last layer to match desired output class
num_classes = 2
in_ftrs = sq_model.classifier[1].in_channels
features = list(sq_model.classifier.children())[:-3] # Remove last 3 layers
features.extend([nn.Conv2d(in_ftrs, num_classes, kernel_size=1)]) # Add
features.extend([nn.ReLU(inplace=True)]) # Add
features.extend([nn.AdaptiveAvgPool2d(output_size=(1,1))]) # Add
sq_model.classifier = nn.Sequential(*features)

In [25]:
model.state_dict()

OrderedDict([('features.0.0.weight',
              tensor([[[[-5.4401e-01, -3.7206e-02,  6.3111e-01],
                        [-1.0392e+00, -1.3085e-01,  1.2488e+00],
                        [-1.0812e+00, -1.8459e-01,  1.1340e+00]],
              
                       [[-8.5868e-01, -1.1450e-01,  8.6298e-01],
                        [-1.4365e+00, -1.9233e-01,  1.6726e+00],
                        [-1.4089e+00, -1.5873e-01,  1.6051e+00]],
              
                       [[-2.8959e-01, -5.9951e-02,  3.3583e-01],
                        [-6.4620e-01, -3.3095e-02,  9.3563e-01],
                        [-7.2531e-01, -2.2003e-01,  7.5415e-01]]],
              
              
                      [[[ 9.2411e-02,  8.2445e-02, -8.0609e-02],
                        [ 1.7470e-01, -3.9439e-02, -1.4034e-01],
                        [ 1.1751e-01, -3.8109e-02, -1.8781e-01]],
              
                       [[-2.5695e-02, -3.3787e-01, -2.9581e-01],
                        [-3.5672e-01, 

In [26]:
last_layer_list_1 = [model.state_dict()['classifier.1.weight'], model.state_dict()['classifier.1.bias']]

In [34]:
weights = last_layer_list_1[0]
weights.size()

torch.Size([2, 1280])

In [32]:
last_layer_list_2 = [sq_model.state_dict()['classifier.1.weight'], sq_model.state_dict()['classifier.1.bias']]

In [35]:
weights2 = last_layer_list_2[0]
weights2.size()

torch.Size([2, 512, 1, 1])

In [39]:
model3 = models.shufflenet_v2_x0_5(weights=ShuffleNet_V2_X0_5_Weights.DEFAULT)

In [40]:
print(f"The squeezenet model has {count_parameters(model3):,} trainable parameters")

The squeezenet model has 1,366,792 trainable parameters


In [41]:
model3

ShuffleNetV2(
  (conv1): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (stage2): Sequential(
    (0): InvertedResidual(
      (branch1): Sequential(
        (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
      )
      (branch2): Sequential(
        (0): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_

In [42]:
model3.fc = nn.Linear(in_features=model3.fc.in_features, out_features=2)

In [43]:
model3

ShuffleNetV2(
  (conv1): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (stage2): Sequential(
    (0): InvertedResidual(
      (branch1): Sequential(
        (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
      )
      (branch2): Sequential(
        (0): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_

In [45]:
model3.state_dict()

OrderedDict([('conv1.0.weight',
              tensor([[[[ 2.0409e-01,  6.9742e-01,  2.6604e-01],
                        [ 8.3504e-01,  1.5789e+00,  9.5059e-01],
                        [ 5.9619e-01,  1.1951e+00,  6.4707e-01]],
              
                       [[-3.5114e-01, -4.9865e-01, -4.5947e-01],
                        [-5.9829e-01, -7.5144e-01, -6.8014e-01],
                        [-7.3751e-01, -8.5755e-01, -8.1673e-01]],
              
                       [[ 2.5052e-01, -2.1778e-01,  2.6758e-01],
                        [-2.3854e-01, -8.9849e-01, -2.6572e-01],
                        [ 1.3435e-01, -3.9654e-01,  1.4120e-01]]],
              
              
                      [[[ 1.9340e-03,  3.6676e-03,  2.4809e-03],
                        [ 3.0431e-04,  2.5875e-03,  2.8172e-03],
                        [ 5.9309e-04,  3.0862e-03,  3.9268e-03]],
              
                       [[ 2.5816e-03,  3.5265e-03,  1.9591e-03],
                        [ 1.2624e-03,  2.88

In [ ]:
model4 = torch.hub.load("coderx7/simplenet_pytorch:v1.0.0", "simplenetv1_small_m2_05", pretrained=True)
model4

/home/anas/anaconda3/envs/pylinenv/lib/python3.10/site-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/coderx7/simplenet_pytorch/zipball/v1.0.0" to /home/anas/.cache/torch/hub/v1.0.0.zip
